# Build Simple Tool-Use AI Agents in LangGraph

Here we will extend the capability of the previously built Augmented LLM with feedback from the tool execution back to the LLM node to process it and generate human-like answers to user queries.

### Tool-based Agentic AI System

- Dynamic Decision-Making: LLM determines whether to directly respond or invoke a tool based on the query context.
- Seamless Tool Integration: External tools are integrated to handle specific tasks, such as real-time web queries or computations.
- Workflow Flexibility: Conditional routing ensures efficient task delegation:
  - Tool Required: Routes to tool execution.
  - No Tool Required: Ends the workflow with an LLM response.
- Feedback Loop: Incorporates a feedback loop to improve responses by combining LLM insights and tool outputs to further improve responses or call more tools if needed

![](https://i.imgur.com/DHxiOLl.png)


In [0]:
!pip install -qqqq langchain==0.3.14
!pip install -qqqq langchain-openai==0.3.0
!pip install -qqqq langchain-community==0.3.14
!pip install -qqqq langgraph==0.2.64

In [0]:
%pip install -U -qqqq mlflow databricks-langchain pydantic databricks-agents unitycatalog-langchain[databricks]
dbutils.library.restartPython()

In [0]:
from getpass import getpass
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph.message import add_messages
from databricks_langchain.uc_ai import (
    DatabricksFunctionClient,
    UCFunctionToolkit,
    set_uc_function_client,
)
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from databricks_langchain import ChatDatabricks

## State

First, define the [State](https://langchain-ai.github.io/langgraph/concepts/low_level/#state) of the graph.

The State schema serves as the input schema for all Nodes and Edges in the graph.

Let's use the `TypedDict` class from python's `typing` module as our schema, which provides type hints for the keys.

In [0]:
class State(TypedDict):
    messages: Annotated[list, add_messages]

## Augment the LLM with tools

Here we define our custom search tool and then bind it to the LLM to augment the LLM

In [0]:
uc_client = DatabricksFunctionClient()
set_uc_function_client(uc_client)

LLM_ENDPOINT = "databricks-meta-llama-3-3-70b-instruct"
llm = ChatDatabricks(endpoint=LLM_ENDPOINT)

catalog = "agentic_ai"
schema = "databricks"

uc_tool_names = [f"{catalog}.{schema}.search_web"]
uc_toolkit = UCFunctionToolkit(function_names=uc_tool_names)
tools=[*uc_toolkit.tools]
llm_with_tools = llm.bind_tools(tools=tools)

In [0]:
llm_with_tools.invoke('what is the latest news on nvidia')

## Create the Graph with the Tool-Use Agentic System

![](https://i.imgur.com/DHxiOLl.png)

In [0]:
from langgraph.graph import StateGraph, START, END
from typing import Annotated, Literal

# from langgraph.prebuilt import tools_condition

# Custom tools_condition function
def tools_condition(state: State) -> Literal["tools", END]:
    """Route to tools if the last message has tool calls, otherwise end."""
    messages = state["messages"]
    last_message = messages[-1]
    
    # Check if the last message has tool calls
    if hasattr(last_message, 'tool_calls') and last_message.tool_calls:
        return "tools"
    return END

# Custom Tool Node function
def tool_node(state: State) -> State:
    """Execute tools based on tool calls in the last message"""
    messages = state["messages"]
    last_message = messages[-1]
    
    tool_calls = last_message.tool_calls if hasattr(last_message, 'tool_calls') else []
    
    tool_messages = []
    for tool_call in tool_calls:
        tool_name = tool_call["name"]
        tool_args = tool_call["args"]
        
        # Find and execute the tool
        tool = next((t for t in tools if t.name == tool_name), None)
        if tool:
            result = tool.invoke(tool_args)
            tool_messages.append({
                "role": "tool",
                "content": str(result),
                "tool_call_id": tool_call["id"]
            })
    
    return {"messages": tool_messages}

# Augmented LLM with Tools Node function
def tool_calling_llm(state: State) -> State:
    current_state = state["messages"]
    return {"messages": [llm_with_tools.invoke(current_state)]}

# Build the graph
builder = StateGraph(State)
builder.add_node("tool_calling_llm", tool_calling_llm)
builder.add_node("tools", tool_node)
builder.add_edge(START, "tool_calling_llm")

# Conditional Edge
builder.add_conditional_edges(
    "tool_calling_llm",
    # If the latest message (result) from LLM is a tool call -> tools_condition routes to tools
    # If the latest message (result) from LLM is a not a tool call -> tools_condition routes to END
    tools_condition,
    ["tools", END]
)
builder.add_edge("tools", "tool_calling_llm") # this is the key feedback loop
builder.add_edge("tools", END)
agent = builder.compile()

In [0]:
agent

In [0]:
user_input = "Explain AI in 2 bullets"
for event in agent.stream({"messages": user_input},
                          stream_mode='values'):
    event['messages'][-1].pretty_print()

In [0]:
user_input = "What is the latest news on OpenAI product releases"
for event in agent.stream({"messages": user_input},
                          stream_mode='values'):
    event['messages'][-1].pretty_print()

In [0]:
event['messages'][-1]

In [0]:
from IPython.display import display, Markdown

display(Markdown(event['messages'][-1].content))